# Entrenamiento de audio con corrupciones
Hecho por:
* Ammi Beltrán
* Fernanda Borja
* Luciano Vidal

**Dirección del dataset**

In [1]:
DATA = "C:\\audio_dataset\\UrbanSound8K"

## Librerías

In [2]:
# Misc
import os
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
# Locals
import audio.audio_corrupts as corrupt
import audio.audio_model as audiotrain
import audio.audio_data as dataudio
# Torch
import torch
import torchaudio
from torch import nn
from torch.utils.data import DataLoader

## Dataset

In [3]:
np.random.seed(0)
file = os.path.join(DATA, "metadata", "UrbanSound8K.csv")
df = pd.read_csv(file)
df['relative_path'] = '\\audio' + '\\fold' + df['fold'].astype(str) + '\\' + df['slice_file_name'].astype(str)
df['corrupt'] = np.random.rand(len(df))
df = df[['relative_path', 'classID', 'corrupt']]
df.head()

,relative_path,classID,corrupt
0,\audio\fold5\100032-3-0-0.wav,3,0.548814
1,\audio\fold5\100263-2-0-117.wav,2,0.715189
2,\audio\fold5\100263-2-0-121.wav,2,0.602763
3,\audio\fold5\100263-2-0-126.wav,2,0.544883
4,\audio\fold5\100263-2-0-137.wav,2,0.423655


In [4]:
# from torch.utils.data import random_split
from sklearn.model_selection import train_test_split
# num_items = len(df)
# num_train = round(num_items * 0.8)
# num_val = num_items - num_train
train, val = train_test_split(df, test_size=0.2,random_state=420)
train, val = train.reset_index(drop = True), val.reset_index(drop = True) 
# generator1 = torch.Generator().manual_seed(420)
# train, val = random_split(df, [num_train, num_val], generator=generator1)
print(val.head())
train_ds = dataudio.SoundDS(train, DATA, transform = True)
val_ds = dataudio.SoundDS(val, DATA, transform = False)
# print(len(train_ds))
# print(val_ds[1])
#
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=250, shuffle=True)
val_dl = torch.utils.data.DataLoader(val_ds, batch_size=250, shuffle=False)

                     relative_path  classID   corrupt
0     \audio\fold5\17578-5-0-2.wav        5  0.646628
1  \audio\fold1\193698-2-0-114.wav        2  0.479139
2     \audio\fold6\66623-4-0-2.wav        4  0.367986
3      \audio\fold1\9031-3-2-0.wav        3  0.048076
4    \audio\fold7\168846-5-1-4.wav        5  0.095380


In [5]:
model = audiotrain.AudioModel()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
train = audiotrain.train(model, 200, train_dl, val_dl, criterion, optimizer, state = "train_params\\filter\\filter_100.pt", name = "filter", dir = "train_params\\filter", device = "cuda")

Epoch nro 101/200
Iter: 1/28, Loss:0.08983296900987625
Iter: 2/28, Loss:0.1526486724615097
Iter: 3/28, Loss:0.08454335480928421
Iter: 4/28, Loss:0.12119925767183304
Iter: 5/28, Loss:0.11376010626554489
Iter: 6/28, Loss:0.1067282184958458
Iter: 7/28, Loss:0.11873901635408401
Iter: 8/28, Loss:0.13100527226924896
Iter: 9/28, Loss:0.11831779778003693
Iter: 10/28, Loss:0.09298635274171829
Iter: 11/28, Loss:0.09576462209224701
Iter: 12/28, Loss:0.11767296493053436
Iter: 13/28, Loss:0.07762251794338226
Iter: 14/28, Loss:0.05981041118502617
Iter: 15/28, Loss:0.11199776083230972
Iter: 16/28, Loss:0.0822235643863678
Iter: 17/28, Loss:0.06465118378400803
Iter: 18/28, Loss:0.08178674429655075
Iter: 19/28, Loss:0.09265106916427612
Iter: 20/28, Loss:0.07917360216379166
Iter: 21/28, Loss:0.07826840877532959
Iter: 22/28, Loss:0.08666980266571045
Iter: 23/28, Loss:0.10228229314088821
Iter: 24/28, Loss:0.0822746753692627
Iter: 25/28, Loss:0.09642067551612854
Iter: 26/28, Loss:0.09991791844367981
Iter: 2